In [1]:
import os
import re
import time
import json
import torch
import random
import codecs
import gensim
import torchtext
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from torch.utils.data import DataLoader

print(f"PyTorch version: {torch.__version__}\nTorchtext version: {torchtext.__version__}")

PyTorch version: 1.7.1
Torchtext version: 0.8.0a0+0f911ec


In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
# torch.backend.cudnn.deterministic = True

In [3]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)
    
    return string.strip().lower()

In [4]:
def build_data(data_path, cv=10):
  
    revs = []
    pos = codecs.open(os.path.join(data_path, "rt-polarity.pos"), "r", encoding='utf-8', errors='ignore').read()
    neg = codecs.open(os.path.join(data_path, "rt-polarity.neg"), "r", encoding='utf-8', errors='ignore').read()
    pos_list = [clean_str(sent) for sent in pos.split('\n')[:-1]]
    neg_list = [clean_str(sent) for sent in neg.split('\n')[:-1]]
    print('pos len', len(pos_list))
    print('neg len', len(neg_list))
    print('total len', len(pos_list) + len(neg_list) )
  
    for sent in pos_list:
        datum = {'label': 1,
                 'text': sent,
                 'num_words': len(sent.split()),
                 'split': np.random.randint(0,cv)
                 }
        revs.append(datum)

    for sent in neg_list:
        datum = {'label': 0,
                 'text': sent,
                 'num_words': len(sent.split()),
                 'split': np.random.randint(0,cv)
                }
        revs.append(datum)

    word_to_idx = {'@pad': 0}

    for sent in pos_list + neg_list:
        for word  in sent.split():
            if word not in word_to_idx:
                word_to_idx[word] = len(word_to_idx)

    print(f'num of total data: {len(revs)}')
    print(f'num of vocab: {len(word_to_idx)}')

    df = pd.DataFrame(revs) 
    df.to_csv(os.path.join(data_path, 'polarity_df.csv'), index=False, encoding='utf-8')
    
    print('save the data')
    
    return revs, word_to_idx

In [6]:
data_path = "data"
revs, word_to_idx = build_data(data_path)

pos len 5331
neg len 5331
total len 10662
num of total data: 10662
num of vocab: 18765
save the data


In [28]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

TEXT   = torchtext.data.Field(sequential=True, tokenize=str.split, batch_first=True, fix_length=56, lower=True)
LABEL  = torchtext.data.LabelField(sequential=False, dtype=torch.float)
FIELDS = [('label', LABEL), ('text', TEXT)]

dataset = torchtext.data.TabularDataset(os.path.join(data_path, "polarity_df.csv"), fields=FIELDS, format='csv', skip_header=True)

train_data, test_data = dataset.split(random_state=random.seed(SEED), split_ratio=0.9)

print('train_data', len(train_data))
print('test_data', len(test_data))

train_data 9596
test_data 1066


/Users/jonghyunlee/miniconda3/envs/nlp/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [29]:
w2v = gensim.models.KeyedVectors.load_word2vec_format(os.path.join(data_path, 'GoogleNews-vectors-negative300.bin.gz'), binary = True)
w2v

In [30]:
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

In [31]:
EMBEDDING_DIM = 300
w2v_vectors = []

for token, idx in tqdm(TEXT.vocab.stoi.items()):
    if token in w2v.wv.vocab.keys():
        w2v_vectors.append(torch.FloatTensor(w2v[token]))
    else:
        w2v_vectors.append(torch.zeros(EMBEDDING_DIM))
#         w2v_vectors.append(torch.distributions.Uniform(-0.25, +0.25).sample((EMBEDDING_DIM, )))

print(len(w2v_vectors))

  0%|          | 0/17863 [00:00<?, ?it/s]

/Users/jonghyunlee/miniconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


17863


In [32]:
# making iterators
train_iterator,  test_iterator = torchtext.data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size=BATCH_SIZE, 
    device=device, 
    sort=False, 
    shuffle = True)

/Users/jonghyunlee/miniconda3/envs/nlp/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [33]:
for a , b in train_iterator:
    print(a)
    print(a.size())
    print()
    print(b)
    print(b.size())
    break

tensor([[  741,     3,  4650,  ...,     1,     1,     1],
        [   42,    38,    56,  ...,     1,     1,     1],
        [ 2735,   982,     8,  ...,     1,     1,     1],
        ...,
        [    4,  4401,  8082,  ...,     1,     1,     1],
        [   10,  2667, 13000,  ...,     1,     1,     1],
        [   30,     2,  1602,  ...,     1,     1,     1]])
torch.Size([64, 56])

tensor([0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0.,
        1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1.,
        1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1.,
        0., 1., 1., 0., 0., 0., 1., 1., 1., 0.])
torch.Size([64])


/Users/jonghyunlee/miniconda3/envs/nlp/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [34]:
import torch.nn as nn
import torch.nn.functional as F

class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [batch size, sent len]
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.permute(0, 2, 1)
        
        #embedded = [batch size, emb dim, sent len]
        conved = [F.tanh(conv(embedded)) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat(pooled, dim = 1))
        
        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [35]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [3, 4, 5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

print('INPUT_DIM', INPUT_DIM)
print('EMBEDDING_DIM', EMBEDDING_DIM)
print('PAD_IDX', PAD_IDX)
print('UNK_IDX', UNK_IDX)

INPUT_DIM 17863
EMBEDDING_DIM 300
PAD_IDX 1
UNK_IDX 0


In [36]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,719,501 trainable parameters


In [37]:
# for check up (before applying word2vec)
test = 'good'
test_indx = TEXT.vocab.stoi[test]
print(f'test string: {test}\nTest string: {test_indx}')


original_vector = w2v[test]
torch_model_vector = model.embedding(torch.tensor([test_indx]))[0]
print()
print(f'before apply\norigianl: {original_vector[:3]}\ntorch_vector: {torch_model_vector[:3]}')

# apply w2v
TEXT.vocab.set_vectors(TEXT.vocab.stoi, w2v_vectors, EMBEDDING_DIM)
pretrained_embeddings = torch.FloatTensor(TEXT.vocab.vectors)
model.embedding.weight.data.copy_(pretrained_embeddings)

# for check up (after applying word2vec)
torch_model_vector = model.embedding(torch.tensor([test_indx]))[0]
print()
print(f'after apply\norigianl: {original_vector[:3]}\ntorch_vector: {torch_model_vector[:3]}')

test string: good
Test string: 54

before apply
origianl: [ 0.04052734  0.0625     -0.01745605]
torch_vector: tensor([ 2.2228, -0.3042, -0.0084], grad_fn=<SliceBackward>)

after apply
origianl: [ 0.04052734  0.0625     -0.01745605]
torch_vector: tensor([ 0.0405,  0.0625, -0.0175], grad_fn=<SliceBackward>)


In [39]:
# padding -> zero vectors
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX]

# unknown token -> randomly initialized with uniform distribution
model.embedding.weight.data[UNK_IDX] = torch.distributions.Uniform(-0.25, +0.25).sample((EMBEDDING_DIM,))
model.embedding.weight.data[UNK_IDX]
print()

In [40]:
optimizer = torch.optim.Adadelta(model.parameters())

# BCEWithLogitsLoss automatically does softmax function
criterion = torch.nn.BCEWithLogitsLoss()

model     = model.to(device)
criterion = criterion.to(device)

In [41]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    
    return acc

In [67]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        #  l2 norm (weight contraints): 3
        with torch.no_grad():
            for param in model.parameters():
                param.clamp_(min=-3, max=3)
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    
    return elapsed_mins, elapsed_secs

In [46]:
N_EPOCHS = 10
best_test_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    test_loss, test_acc = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), os.path.join('weights', 'latest_weigths.pt'))
    
    print(f'Epoch: {epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
    print(f'\t Test. Loss: {test_loss:.3f} |  Val. Acc: {test_acc * 100:.2f}%')

Epoch: 01 | Epoch Time: 0m 26s
	Train Loss: 0.276 | Train Acc: 88.63%
	 Test. Loss: 0.436 |  Val. Acc: 80.31%
Epoch: 02 | Epoch Time: 0m 26s
	Train Loss: 0.199 | Train Acc: 92.55%
	 Test. Loss: 0.466 |  Val. Acc: 79.44%
Epoch: 03 | Epoch Time: 0m 26s
	Train Loss: 0.131 | Train Acc: 95.38%
	 Test. Loss: 0.514 |  Val. Acc: 78.79%
Epoch: 04 | Epoch Time: 0m 25s
	Train Loss: 0.075 | Train Acc: 97.97%
	 Test. Loss: 0.556 |  Val. Acc: 79.90%
Epoch: 05 | Epoch Time: 0m 25s
	Train Loss: 0.043 | Train Acc: 99.06%
	 Test. Loss: 0.654 |  Val. Acc: 78.48%
Epoch: 06 | Epoch Time: 0m 25s
	Train Loss: 0.027 | Train Acc: 99.51%
	 Test. Loss: 0.654 |  Val. Acc: 79.11%
Epoch: 07 | Epoch Time: 0m 25s
	Train Loss: 0.016 | Train Acc: 99.80%
	 Test. Loss: 0.660 |  Val. Acc: 80.49%
Epoch: 08 | Epoch Time: 0m 25s
	Train Loss: 0.010 | Train Acc: 99.92%
	 Test. Loss: 0.688 |  Val. Acc: 79.95%
Epoch: 09 | Epoch Time: 0m 25s
	Train Loss: 0.007 | Train Acc: 99.93%
	 Test. Loss: 0.701 |  Val. Acc: 80.18%
Epoch: 10 

In [66]:
model.load_state_dict(torch.load(os.path.join('weights', 'latest_weigths.pt')))
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

tensor([ 4.7737e+00,  5.1443e-03, -4.3022e+00, -1.1437e+00, -1.9547e+00,
         1.3992e+00, -9.0374e-01,  2.9036e+00,  2.9538e+00, -2.3983e+00,
         2.2909e+00, -1.9025e+00, -3.0645e+00,  3.7888e-01,  4.3808e+00,
        -5.7399e+00, -5.0785e+00,  3.5388e+00, -4.4906e+00, -5.0073e+00,
        -3.7470e+00,  5.3025e+00, -5.0526e+00,  5.4262e+00,  6.4762e-01,
         6.8045e+00,  1.1273e-01,  4.8109e-01,  8.0179e+00, -4.0152e-01,
         5.8624e-01,  6.3370e+00, -8.7900e-01, -2.7857e+00, -5.6614e-01,
         6.0901e+00,  1.1505e+00, -7.5684e-01, -2.1399e+00, -2.5490e+00,
         3.8377e+00, -2.7731e+00,  1.3195e+00, -1.7095e+00,  1.9329e+00,
        -1.6782e+00,  1.2829e+00,  2.6101e+00,  7.9525e-01,  3.7645e+00,
         1.3969e+00,  6.9994e+00, -1.1810e-01, -5.6805e+00,  1.8092e-01,
         2.2383e+00,  5.4572e+00,  2.4550e+00,  1.5839e+00,  1.8229e+00,
        -4.5187e+00, -3.4050e+00,  4.3651e+00, -2.8964e+00])
tensor([-3.4420, -0.0412, -0.9857,  1.1341, -3.6155,  3.3118,  

tensor([-0.8456, -0.3221,  0.9992,  2.0143,  1.1635,  1.0027, -0.6904,  4.3192,
        -0.7128, -1.5601,  0.5725,  1.8753, -0.2105,  2.6226, -2.9754,  4.8488,
        -4.8531, -8.3488,  4.2852, -6.5219,  0.6712, -1.8527, -1.7970, -1.8057,
         1.2812, -0.7504, -3.1256, -7.1719,  2.8686, -2.3451,  0.0779,  5.6151,
         2.9809, -0.7065,  2.1750, -2.5555, -4.7568,  1.7967, -2.2392,  2.2917,
         0.5784,  4.6005,  4.6674,  1.3962, -4.9643, -2.9012, -1.3785, -1.2938,
         0.8299, -0.8498,  1.0485, -2.5579,  3.3810, -1.4089, -1.8781,  2.2435,
         0.2126, -1.1327, -1.8017, -1.8398,  1.9083,  1.3942,  0.7747,  1.6275])
Test Loss: 0.434 | Test Acc: 80.46%


In [62]:
my_sentence = "this film is terrible"

for word in my_sentence.split():
    

this
film
is
terrible
